In [93]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import svm 
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from collections import defaultdict
import itertools

In [49]:
df = pd.read_csv("../data/run-over-dataset.csv")
df.head()

,VERBALE,DATA,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,...,II raggio sx.1,III raggio sx.1,IV raggio sx.1,V raggio sx.1,Art. coxo-femorale dx,Art. coxo-femorale sx,Rotula o Ginocchio dx,Rotula o Ginocchio sx,Caviglia dx,Caviglia sx
0,85567,10/29/1999,0,81,84,"1,75","27,4285714285714",0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,85829,1/14/2000,1,69,69,"1,62","26,291723822588",1,4,4,...,0,0,0,0,0,0,0,0,0,0
2,85977,3/10/2000,1,71,67,"1,55","27,8876170655567",1,2,0,...,0,0,0,0,0,0,0,0,0,0
3,86220,6/14/2000,1,54,60,"1,59","23,7332384003797",1,4,0,...,0,0,0,0,0,0,0,0,0,0
4,86247,6/22/2000,1,78,69,"1,67","24,7409372871024",1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
cols_to_drop = ['VERBALE', 'DATA', 'BMI']
X = df.drop(columns=cols_to_drop)
X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X.head()

,SESSO,ANNI,PESO,ALTEZZA,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,Testa:Cervelletto,Testa:Tronco encefalico,...,II raggio sx.1,III raggio sx.1,IV raggio sx.1,V raggio sx.1,Art. coxo-femorale dx,Art. coxo-femorale sx,Rotula o Ginocchio dx,Rotula o Ginocchio sx,Caviglia dx,Caviglia sx
0,0,81,84,175,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,69,69,162,1,4,4,4,4,4,...,0,0,0,0,0,0,0,0,0,0
2,1,71,67,155,1,2,0,1,1,2,...,0,0,0,0,0,0,0,0,0,0
3,1,54,60,159,1,4,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,78,69,167,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
X.shape

(130, 364)

In [52]:
def low_std_cols(X, beta):
    columns_low_std = []
    for column in X.columns:
        if X[column].std() <= beta:
            columns_low_std.append(column)

    return columns_low_std, len(columns_low_std)

In [53]:
def zero_columns(X):
    zero_cols = []
    cols, n = low_std_cols(X, 0)

    for col in cols:
        if sum(X[col]) == 0:
            zero_cols.append(col)
    
    return zero_cols, (len(zero_cols) == n)

In [54]:
cols_to_drop, ok = zero_columns(X)
X = X.drop(columns=cols_to_drop)
X.shape

(130, 324)

Eliminate 40 colonne composte da soli 0 e per cui ininfluenti per la classificazione.

In [55]:
def equal_columns(X):
    cols = set()

    for column1 in X.columns:
        for column2 in X.columns:
            equals = X[column1] == X[column2]
            if (equals.all() and column1 != column2):
                cols.add((column1, column2))

    return cols

cols = equal_columns(X)
cols

{('Gomito sx', 'Polso sx'),
 ('Linea angolo-scapolare dx.10', 'Linea angolo-scapolare dx.9'),
 ('Linea angolo-scapolare dx.9', 'Linea angolo-scapolare dx.10'),
 ('Linea ascellare anteriore dx.10', 'Linea ascellare anteriore dx.11'),
 ('Linea ascellare anteriore dx.11', 'Linea ascellare anteriore dx.10'),
 ('Linea ascellare anteriore dx.8', 'Linea ascellare anteriore dx.9'),
 ('Linea ascellare anteriore dx.9', 'Linea ascellare anteriore dx.8'),
 ('Linea ascellare media dx.10', 'Linea ascellare media dx.11'),
 ('Linea ascellare media dx.11', 'Linea ascellare media dx.10'),
 ('Linea ascellare media sx.10', 'Linea ascellare media sx.9'),
 ('Linea ascellare media sx.9', 'Linea ascellare media sx.10'),
 ('Linea ascellare posteriore dx.2', 'Linea ascellare posteriore dx.3'),
 ('Linea ascellare posteriore dx.3', 'Linea ascellare posteriore dx.2'),
 ('Linea emiclaveare dx.10', 'Linea emiclaveare dx.11'),
 ('Linea emiclaveare dx.11', 'Linea emiclaveare dx.10'),
 ('Linea parasternale dx.10', 'Lin

In [56]:
X = X.T.drop_duplicates().T
X.shape

(130, 312)

Eliminate 12 colonne duplicate.

In [60]:
mezzo_1 = X[X['Mezzo'] == 1].drop(columns='Mezzo').values
mezzo_0 = X[X['Mezzo'] == 0].drop(columns='Mezzo').values
classes = X['Mezzo'].values

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0])

In [117]:
def nested_cv_svm_accuracy(X, Y):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-3, 2, 10)
    nus = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    random_seeds = np.random.randint(20, 100000, size=20)
    for seed in random_seeds:
        outer_cv = KFold(n_splits=10, shuffle=True, random_state=seed)
        best_accuracy = 0

        for trainval_idx, test_idx in outer_cv.split(X):
            X_trainval = X[trainval_idx]
            X_test = X[test_idx]

            inner_cv = KFold(n_splits=7, shuffle=True, random_state=seed)
            for train_idx, valid_idx in inner_cv.split(X_trainval):
                X_train = X_trainval[train_idx]
                X_valid = X_trainval[valid_idx]

                params_list = list(itertools.product(kernels, gammas, nus))
                for params in params_list:
                    clf = svm.OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])
                    clf.fit(X_train)

                    positive_class_predictions = clf.predict(X_valid)
                    negative_class_predictions = clf.predict(Y)

                    true_values = np.concatenate((np.full_like(positive_class_predictions, 1), np.full_like(negative_class_predictions, -1)))
                    predicted_values = np.concatenate((positive_class_predictions, negative_class_predictions))

                    accuracy = accuracy_score(true_values, predicted_values)
                    if accuracy> best_accuracy:
                        best_accuracy = accuracy
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]
                    
            clf = svm.OneClassSVM(**best_params)
            clf.fit(X_trainval)

            positive_class_predictions = clf.predict(X_test)
            negative_class_predictions = clf.predict(Y)
            
            tp = sum([1 if pred == 1 else 0 for pred in positive_class_predictions])
            fp = sum([1 if pred == 1 else 0 for pred in negative_class_predictions])

            true_values = np.concatenate((np.full_like(positive_class_predictions, 1), np.full_like(negative_class_predictions, -1)))
            predicted_values = np.concatenate((positive_class_predictions, negative_class_predictions))

            accuracy = accuracy_score(true_values, predicted_values)
            precision = tp / (tp + fp) if (tp + fp) != 0 else 0
            recall = recall_score(true_values, predicted_values)
            f1 = f1_score(true_values, predicted_values)

            accuracy_scores.append(accuracy)
            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

    results = {'best kernel': best_params['kernel'],
               'best gamma': best_params['gamma'],
               'best nu': best_params['nu'],
               'score used for model selection': 'accuracy score',
               'accuracy score mean': np.mean(accuracy_scores) * 100,
               'accuracy score std': np.std(accuracy_scores) * 100,
               'precision score mean': np.mean(precision_scores) * 100,
               'precision score std': np.std(precision_scores) * 100,
               'recall score mean': np.mean(recall_scores) * 100,
               'recall score std': np.std(recall_scores) * 100,
               'f1 score mean': np.mean(f1_scores) * 100,
               'f1 score std': np.std(f1_scores) * 100}

    return results

In [118]:
results = nested_cv_svm_accuracy(mezzo_0, mezzo_1)

In [122]:
results['data set'] = 'no 0 columns, no equal columns'

In [123]:
scores_df = pd.DataFrame(results, index=[0])
scores_df

,best kernel,best gamma,best nu,score used for model selection,accuracy score mean,accuracy score std,precision score mean,precision score std,recall score mean,recall score std,f1 score mean,f1 score std,data set
0,rbf,0.012915,0.01,accuracy score,89.395522,0.849534,0.57619,4.106184,0.571429,4.247448,0.567537,4.112591,"no 0 columns, no equal columns"


In [124]:
def nested_cv_svm_f1(X, Y):
    kernels = ['linear', 'rbf']
    gammas = np.logspace(-3, 2, 10)
    nus = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    best_params = {'kernel': '', 'gamma': 0, 'nu': 0}

    random_seeds = np.random.randint(20, 100000, size=20)
    for seed in random_seeds:
        outer_cv = KFold(n_splits=10, shuffle=True, random_state=seed)
        best_f1 = 0

        for trainval_idx, test_idx in outer_cv.split(X):
            X_trainval = X[trainval_idx]
            X_test = X[test_idx]

            inner_cv = KFold(n_splits=7, shuffle=True, random_state=seed)
            for train_idx, valid_idx in inner_cv.split(X_trainval):
                X_train = X_trainval[train_idx]
                X_valid = X_trainval[valid_idx]

                params_list = list(itertools.product(kernels, gammas, nus))
                for params in params_list:
                    clf = svm.OneClassSVM(kernel=params[0], gamma=params[1], nu=params[2])
                    clf.fit(X_train)

                    positive_class_predictions = clf.predict(X_valid)
                    negative_class_predictions = clf.predict(Y)

                    true_values = np.concatenate((np.full_like(positive_class_predictions, 1), np.full_like(negative_class_predictions, -1)))
                    predicted_values = np.concatenate((positive_class_predictions, negative_class_predictions))

                    f1 = f1_score(true_values, predicted_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['kernel'] = params[0]
                        best_params['gamma'] = params[1]
                        best_params['nu'] = params[2]
                    
            clf = svm.OneClassSVM(**best_params)
            clf.fit(X_trainval)

            positive_class_predictions = clf.predict(X_test)
            negative_class_predictions = clf.predict(Y)
            
            tp = sum([1 if pred == 1 else 0 for pred in positive_class_predictions])
            fp = sum([1 if pred == 1 else 0 for pred in negative_class_predictions])

            true_values = np.concatenate((np.full_like(positive_class_predictions, 1), np.full_like(negative_class_predictions, -1)))
            predicted_values = np.concatenate((positive_class_predictions, negative_class_predictions))

            accuracy = accuracy_score(true_values, predicted_values)
            precision = tp / (tp + fp) if (tp + fp) != 0 else 0
            recall = recall_score(true_values, predicted_values)
            f1 = f1_score(true_values, predicted_values)

            accuracy_scores.append(accuracy)
            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

    results = {'best kernel': best_params['kernel'],
               'best gamma': best_params['gamma'],
               'best nu': best_params['nu'],
               'score used for model selection': 'f1 score',
               'accuracy score mean': np.mean(accuracy_scores) * 100,
               'accuracy score std': np.std(accuracy_scores) * 100,
               'precision score mean': np.mean(precision_scores) * 100,
               'precision score std': np.std(precision_scores) * 100,
               'recall score mean': np.mean(recall_scores) * 100,
               'recall score std': np.std(recall_scores) * 100,
               'f1 score mean': np.mean(f1_scores) * 100,
               'f1 score std': np.std(f1_scores) * 100}

    return results

In [125]:
results = nested_cv_svm_f1(mezzo_0, mezzo_1)

In [129]:
results['data set'] = 'no 0 columns, no equal columns'
new_results = pd.DataFrame(results, index=[0])
scores_df = pd.concat([scores_df, new_results], ignore_index=True)
scores_df

,best kernel,best gamma,best nu,score used for model selection,accuracy score mean,accuracy score std,precision score mean,precision score std,recall score mean,recall score std,f1 score mean,f1 score std,data set
0,rbf,0.012915,0.01,accuracy score,89.395522,0.849534,0.576190,4.106184,0.571429,4.247448,0.567537,4.112591,"no 0 columns, no equal columns"
1,rbf,0.001000,0.01,f1 score,71.276119,7.239187,15.507375,7.617038,42.357143,24.759146,21.732711,10.306257,"no 0 columns, no equal columns"
